In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

# import _utils # amanda's collected utils

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import datetime as dt
import numpy as np

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"

import intake

catalog = intake.open_catalog("./metrics_catalog.yml")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_289/529163112.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  imp

# Safety Metric Demo 
Testing on Watsonville-Santa Cruz multimodal corridor project along SR-1

### Demo Crash Reduction Factors:
* Transit signal priority: 14% [cmf clearinghouse reference](https://www.cmfclearinghouse.org/detail.php?facid=11233)
* Mode separation: 41% [ref](https://www.cmfclearinghouse.org/detail.php?facid=2146)
* crosswalks/flashing beacons: 15% [ref](https://www.cmfclearinghouse.org/detail.php?facid=2917)
* Aux lane: 23% [ref](https://www.cmfclearinghouse.org/detail.php?facid=3899)

In [2]:
# combine crash reduction factors - toy example
# reference: https://www.cmfclearinghouse.org/collateral/Combining_Multiple_CMFs_Final.pdf 
# CCRFi = 1 – [(1-CRF1i)*(1-CRF2i)*(1-CRF3i)] 

CRF = 1-((1-0.14)*(1-0.41)*(1-0.15)*(1-0.23))

In [3]:
CRF

0.6679067

## Version 2: mode separated location
This location data is hand-drawn

In [4]:
# repeatable function
def readmode(catname: str):
    # projloc = catalog.catname.read()
    item = getattr(catalog, catname)
    projloc = item.read()
    projloc = projloc.to_crs(shared_utils.geography_utils.CA_NAD83Albers)
    projloc['widget'] = f'{catname}'
    projloc['b100'] = projloc.buffer(30)
    projloc = projloc.set_geometry('b100')
    return projloc  

In [5]:
bikeped = readmode('shp_demoproj_bikeped')
auxlane = readmode('shp_demoproj_auxlane')
busshoulder = readmode('shp_demoproj_busshoulder')
multimodal = readmode('shp_demoproj_multimodal')

In [6]:
all_proj_loc = pd.concat([bikeped, auxlane, busshoulder, multimodal])

In [8]:
# read in all severity from TIMS
tims = gpd.read_parquet('gs://calitp-analytics-data/data-analyses/safety_projects/tims_all_severity.parquet')

In [13]:
# count crashes in project area
tims_pclip = tims.clip(all_proj_loc)
tims_pclip = tims_pclip.assign(ped_crash = np.where(tims_pclip['PEDESTRIAN_ACCIDENT']=="Y",1,0),
                             bike_crash = np.where(tims_pclip['BICYCLE_ACCIDENT']=="Y",1,0),
                               fsi_crash = np.where(tims_pclip['COLLISION_SEVERITY']<=2,1,0)
                            )
tims_pclip['pedbike_crash'] = tims_pclip[["ped_crash", "bike_crash"]].max(axis=1)

In [10]:
# all crashes 
len(tims_pclip)

258

In [14]:
# sum pedbike
tims_pclip >> count(_.fsi_crash)

,fsi_crash,n
0,0,238
1,1,20


In [18]:
tims_pclip >> head(20)

,CASE_ID,ACCIDENT_YEAR,COLLISION_DATE,COLLISION_TIME,COLLISION_SEVERITY,PCF_VIOL_CATEGORY,TYPE_OF_COLLISION,MVIW,NUMBER_KILLED,NUMBER_INJURED,...,BICYCLE_ACCIDENT,LATITUDE,LONGITUDE,POINT_X,POINT_Y,geometry,ped_crash,bike_crash,fsi_crash,pedbike_crash
844789,90757712,2018,2018-06-22,1213,4,08,B,C,0,1,...,,36.975269,-121.902817,-121.902824,36.975258,POINT (-169148.791 -114035.104),0,0,0,0
845287,91031815,2019,2019-07-11,915,4,03,C,C,0,1,...,,36.975349,-121.903427,-121.903435,36.975330,POINT (-169202.878 -114025.960),0,0,0,0
840672,90241999,2016,2016-08-10,1020,4,04,C,C,0,2,...,,36.975330,-121.903600,-121.903597,36.975352,POINT (-169217.286 -114023.281),0,0,0,0
841949,90479062,2017,2017-06-14,1905,4,03,C,C,0,1,...,,36.975410,-121.904170,-121.904178,36.975344,POINT (-169268.907 -114023.067),0,0,0,0
845824,91361298,2020,2020-12-02,650,4,03,C,C,0,1,...,,36.975342,-121.903999,-121.903992,36.975399,POINT (-169252.227 -114017.336),0,0,0,0
841888,90420239,2017,2017-03-16,2215,4,07,B,C,0,2,...,,36.975430,-121.904040,-121.904043,36.975405,POINT (-169256.773 -114016.586),0,0,0,0
845831,91367541,2020,2020-12-06,1700,4,08,E,I,0,2,...,,36.975441,-121.905060,-121.905052,36.975483,POINT (-169346.297 -114006.115),0,0,0,0
845780,91331864,2020,2020-10-12,745,4,03,C,C,0,1,...,,36.975471,-121.904633,-121.904625,36.975494,POINT (-169308.297 -114005.607),0,0,0,0
841957,90488629,2017,2017-06-23,900,4,04,C,C,0,2,...,,36.975410,-121.904680,-121.904661,36.975502,POINT (-169311.486 -114004.713),0,0,0,0
844202,90440227,2017,2017-04-07,730,4,03,C,C,0,3,...,,36.975520,-121.904750,-121.904750,36.975520,POINT (-169319.351 -114002.509),0,0,0,0


In [17]:
# map together
projmap = all_proj_loc.explore(tiles="cartodbpositron", column="widget")
tims_pclip.explore(m=projmap, column="COLLISION_SEVERITY")